<a href="https://colab.research.google.com/github/Amir-h055/Image-Procccessing/blob/main/image_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2

%tensorflow_version 2.x 
# TensorFlow and tf.keras
import tensorflow as tf
import seaborn as sns
from tensorflow import keras
from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_directory = '/content/drive/My Drive/Comp 478 - Project/Chest X-ray (Covid-19 & Pneumonia)/Data/train/*'
test_directory = '/content/drive/My Drive/Comp 478 - Project/Chest X-ray (Covid-19 & Pneumonia)/Data/test/*'

In [ ]:
train_directory = '/content/drive/My Drive/Comp 478 - Project/Chest X-ray (Covid-19 & Pneumonia)/Data/train/*'
test_directory = '/content/drive/My Drive/Comp 478 - Project/Chest X-ray (Covid-19 & Pneumonia)/Data/test/*'

In [ ]:
SIZE = 140
BATCH_SIZE = 64
TARGET_SIZE = (SIZE,SIZE)
EPOCH_NUM = 10

In [ ]:
categories_dict = {
  0: "COVID19",
  1: "NORMAL",
  2: "PNEUMONIA",
  
}

In [ ]:
train_images = []
train_labels = [] 
label = 0

#Importing the training dataset

for directory_path in glob.glob(train_directory):
    print(train_directory)
    counter = 0
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        counter+=1
        img = cv2.imread(img_path, 0)
        img = cv2.resize(img, TARGET_SIZE)
        train_images.append(img)
        train_labels.append(label)
    
    print(f'Training Images For {categories_dict[label]}: {counter} images loaded')
    label +=1

print(f'Total Training Images: {len(train_images)} images loaded')
x_train = np.array(train_images)
y_train = keras.utils.to_categorical(train_labels, 3)

/content/drive/My Drive/Comp 478 - Project/Chest X-ray (Covid-19 & Pneumonia)/Data/train/*
Training Images For COVID19: 460 images loaded
/content/drive/My Drive/Comp 478 - Project/Chest X-ray (Covid-19 & Pneumonia)/Data/train/*
Training Images For NORMAL: 1266 images loaded
/content/drive/My Drive/Comp 478 - Project/Chest X-ray (Covid-19 & Pneumonia)/Data/train/*
Training Images For PNEUMONIA: 3418 images loaded
Total Training Images: 5144 images loaded


In [ ]:
#sns.countplot(x=,data= y_train)
# type(y_train)
df = pd.DataFrame(train_labels)
df.head()
# print(train_labels)
sns.countplot(x=0,data= df)

AttributeError: ignored

In [ ]:
test_images = []
test_labels = []
label = 0

#importing the testing dataset

for directory_path in glob.glob(test_directory):
    counter = 0
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        counter+=1
        img = cv2.imread(img_path, 0)
        img = cv2.resize(img, TARGET_SIZE)
        test_images.append(img)
        test_labels.append(label)
    
    print(f'Testing Images For {categories_dict[label]}: {counter} images loaded')
    label +=1

print(f'Total Testing Images: {len(test_images)} images loaded')
test_images_pred = test_images
test_labels_presentation = test_labels 
test_images = np.array(test_images)
test_labels = keras.utils.to_categorical(test_labels, 3)

In [ ]:
type(train_images)
# convert from integers to floats
x_train = x_train.astype('float32')
test_images_norm = test_images.astype('float32')
# normalize to the range 0-1
x_train /= 255.0
test_images_norm /= 255.0
type(train_images)

In [ ]:
def cnn_model():
# CNN Based Model in Keras
  model = keras.Sequential([
        keras.Input(shape=(140,140,1), name='Original_Images'),
        keras.layers.Conv2D(input_shape=(140,140,1), filters=32, kernel_size=11, 
                            strides=1, activation='relu', name='Conv1'),
        keras.layers.Conv2D(input_shape=(130,130,32), filters=32, kernel_size=11, 
                            strides=1, activation='relu', name='Conv2'),
        keras.layers.MaxPool2D(pool_size=(5, 5), strides=2),
        keras.layers.Conv2D(input_shape=(58,58,32), filters=64, kernel_size=9, 
                            strides=1, activation='relu', name='Conv3'),
        keras.layers.MaxPool2D(pool_size=(5, 5), strides=2),
        keras.layers.Conv2D(input_shape=(23,23,64), filters=128, kernel_size=8, 
                            strides=1, activation='relu', name='Conv4'),
        keras.layers.Conv2D(input_shape=(16,16,128), filters=256, kernel_size=9, 
                            strides=1, activation='relu', name='Conv5'),
        keras.layers.Conv2D(input_shape=(8,8,256), filters=256, kernel_size=8, 
                            strides=1, activation='relu', name='Conv6'),    

        keras.layers.Flatten(),
        keras.layers.Dense(8, activation=tf.keras.activations.relu, name='Dense'),
        keras.layers.Dense(3, activation="softmax")
    ])
  opt = keras.optimizers.Adam(learning_rate=0.005)
  model.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])
  return model

  

In [ ]:
model = cnn_model()

In [ ]:
keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
hist = model.fit(
    x_train,
    y_train,
    epochs=1
)

In [ ]:
df = pd.DataFrame(test_labels_presentation)
df.head()
# print(train_labels)
sns.countplot(x=0,data= df)

In [ ]:
score = model.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def show_image(img, label, guess):
  plt.figure()
  plt.imshow(img, cmap=plt.cm.binary)
  plt.title("Excpected: " + label)
  plt.xlabel("Guess: " + guess)
  plt.colorbar()
  plt.grid(False)
  plt.show()

In [ ]:
def predict(model, image, correct_label):
  prediction = model.predict(np.array([image]))
  print(prediction)
  predicted_class = categories_dict[np.argmax(prediction)]
  show_image(image, categories_dict[correct_label], predicted_class)

In [ ]:
for i in range(len(test_images)):
  image = test_images[i]
  label = np.argmax(test_labels[i])
  predict(model, image, label)